In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from rdkit import Chem
from rdkit.Chem import (
                        PandasTools,
                        Descriptors
                        )

from rdkit.Chem.Draw import IPythonConsole

# Notebook - ML for thermochemical data prediction

We can use the data you worked with for the EDA workshop to demonstrate how fit a simple ML model to predict the boiling point of organic compounds from a small set of descriptors.

This is an example of

**Supervised learning** as applied to a **regression** task.

</br>

Supervised learning
: The model learns from data for which a label (known value or category) is associated with each input (set of feature values)

Regression task
: Model predicts a continuous numerical value based on the input features


In [ ]:
bp_df = pd.read_csv("data/alcohol_acid_phys_data_cleaned.csv")
bp_df

The data is read in from a csv file of the data cleaned in the [EDA workshop](../workshop_files/eda_workshop), so should be prepared, but we can quickly check.

In [ ]:
# Check for missing values
bp_df.isna().sum()

In [ ]:
# Check data types
bp_df.info()

The numerical types are as expected. The category type for the `class` column has not been automatically recognised, understandably. For now we will not be working with that column, so will leave as is.

We will start by dropping any rows without a SMILES string and also the melting point column as we are going to try to predict the boiling point.

In [ ]:
# Drop rows with missing SMILES string
bp_df = bp_df[bp_df["SMILES"] != "not found"].reset_index(drop=True)

bp_df = bp_df.drop(columns=["mp / dC"])
bp_df

In [ ]:
# Add RDKit molecules to the dataframe
PandasTools.AddMoleculeColumnToFrame(bp_df, smilesCol="SMILES")
bp_df

In [ ]:
# Adapted from https://greglandrum.github.io/rdkit-blog/posts/2022-12-23-descriptor-tutorial.html

def getMolDescriptors(mol, descriptor_list=None, missingVal=None):
    ''' calculate the full list of descriptors for a molecule
    
        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    if not(descriptor_list):
        descriptors = Descriptors._descList
    # TODO: Add else clause to handle a list numbers corresponding to the descriptor indices
    else:
        descriptors = [Descriptors._descList[idx] for idx in descriptor_list]

    for nm,fn in descriptors:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

In [ ]:
# These are the descriptors selected to calculate for the molecules.
# 118 NumHAcceptors
# 119 NumHDonors
# 27 BalabanJ - a topological descriptor expressing molecular connectivity and branching
# 28 BertzCT - a topological complexity index
# 83 TPSA - total polar surface area

descriptor_list = [118, 119, 27, 28, 83]


calc_descriptors = [getMolDescriptors(mol, descriptor_list=descriptor_list) for mol in bp_df["ROMol"]]

# Create a dataframe from the calculated descriptors
descriptor_df = pd.DataFrame(calc_descriptors)

# Add the descriptors to the dataframe as new columns
bp_df = pd.concat([bp_df, descriptor_df], axis=1)
bp_df

In [ ]:
# To remind us of the relationship between the variables, we can plot a pairplot and heatmap
# Pairplot
sns.pairplot(bp_df, hue="Class", diag_kind="kde",  palette="viridis")
plt.show()

In [ ]:
corr = bp_df.drop(columns=["Class", "IUPAC name", "SMILES", "ROMol"]).corr()

sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
# corr_abs = bp_df.drop(columns=["Class", "IUPAC name", "SMILES", "ROMol"]).corr().abs()
# upper = corr_abs.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
# upper

This is all looking rather busy - it would be better to look at subsets of the features.

What we can see is:

- The feature most strongly correlated with the target variable is the molecular weight.
- Molecular weight is very strongly correlated with number of carbons and hydrogens
- Most of the other features are only moderately or weakly correlated to the target variable.



In [ ]:
# A closer look at the correlation between the boiling point and the molecular weight
sns.regplot(data=bp_df, x="Molweight g/mol", y="bp / dC",  fit_reg=True,  ci=None)
plt.show()

For the moment, we will drop the `#C` and `#H` columns. We will see that analysing the initial model can help tell us about the importance of the features. 

We will also drop the non-numerical features for this task.

In [ ]:
cols_drop = ["#C", "#H", "IUPAC name", "SMILES", "ROMol", "Class"]

prep_df = bp_df.drop(columns=cols_drop)
prep_df

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# By convention, the target variable is denoted by y and the features are denoted by X

y_full = prep_df["bp / dC"]
X_full = prep_df.drop(columns=["bp / dC"])

In [ ]:
# train_test_split shuffles the data by default and splits it into training and testing sets. The 
# proportion of the data that is used for testing is determined by the test_size parameter. Here, 
# we are using 80 % of the data for training and 20% for the test set. 
# The random_state parameter is used to set the seed for the random number generator so that the 
# results are reproducible.

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

In [ ]:
# Check the size of the training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

[`Scikit-learn`](https://scikit-learn.org/stable/index.html) makes many models available via a consistent interface.

We are going to use a linear regression model for this task.

In [ ]:
from sklearn import linear_model

In [ ]:
# Create a linear regression model
reg = linear_model.LinearRegression()

# Fit the model to the training data
reg.fit(X_train, y_train)

In [ ]:
# Predict the boiling points of the test set
y_pred = reg.predict(X_test)


We can plot the boiling points that the model predicted for the test set against the known true values to see how good a job the model makes of the predictions

In [ ]:

plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '-', lw=1, color="red")
plt.xlabel("True Values [bp / dC]")
plt.ylabel("Predictions [bp / dC]")
plt.show()

In [ ]:
# The r^2 value is a measure of how well the model fits the data. It ranges from 0 to 1, 
# with 1 indicating a perfect fit.
r2 = reg.score(X_test, y_test)
r2

If we look back, we can see that the Pearson correlation coefficient for the relationship between molecular weight and boiling point was 0.84, so the model predicts more accurately than using the molecular weight alone.

The model coefficients tell us about the weighting of the features used by the fitted model.

In [ ]:
print(X_full.columns)
reg.coef_

However, because the magnitude of the features' values are on different scales, the coefficients also incorporate the different scales.

A scaler can be used to transform the features to a consistent scale. Here's we'll use a [`MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) to transform the features to have a scale between 0 and 1.


In [ ]:
# Split the scaled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

# create a min/max scaler
scaler = MinMaxScaler()

# fit the scaler to the training data
scaler.fit(X_train)

# transform the training and testing data separately
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [ ]:
# Create a linear regression model
reg = linear_model.LinearRegression()

# Fit the model to the training data
reg.fit(scaled_X_train, y_train)

# Predict the boiling points of the test set
y_pred = reg.predict(scaled_X_test)

In [ ]:
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '-', lw=1, color="red")
plt.xlabel("True Values [bp / dC]")
plt.ylabel("Predictions [bp / dC]")
plt.show()

In [ ]:
# calculate the R^2 score
r2 = reg.score(scaled_X_test, y_test)
r2

The model's predictions look the same as before, but we can now look at the coefficients.

In [ ]:
print(X_full.columns)
reg.coef_

We can now see that the coefficients which represent the weights of the features in the fitted model indicate that molecular weight - as expected - and density are contributing most strongly to the model